# Optimizando un modelo de Keras para producción

Vamos a ver algunas de las utilidades para mejorar la latencia y el tamaño que ocupan los modelos de Keras. Para ello usaremos la librería `tensorflow-model-optimization` por lo que será necesario instalarla primero con pip.

```bash
$ pip install tensorflow-model-optimization
```

Empezamos importando las librerías necesarias

In [1]:
import os
from pathlib import Path
import time

import numpy as np
import tensorflow as tf
import tensorflow_model_optimization as tfmot

Ahora declaramos una serie de constantes que usaremos a lo largo del ejercicio

In [2]:
BATCH_SIZE = 128
EPOCHS = 10

También aprovecharemos y crearemos una función para ver el tamaño que ocupan los modelos en memoria. Como dependiendo del formato, los modelos se salvarán en un directorio o en un único fichero, la función mirará esos dos casos.

In [3]:
def model_size(path):
    """Returns the size of the model located in path in MiB."""
    if Path(path).is_dir():
        size = sum(
            f.stat().st_size
            for f in Path(path).glob('**/*')
            if f.is_file()
        )
    else:
        size = Path(path).stat().st_size
    
    return size / 1024**2

Para hacer las pruebas usaremos el manido MNIST. Crearemos primero los conjuntos de entrenamiento y test:

In [4]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train / 255.0
X_test = X_test / 255.0

Por último, crearemos el modelo base sobre el que trabajaremos

In [5]:
model = tf.keras.Sequential([
  tf.keras.layers.InputLayer(input_shape=(28, 28)),
  tf.keras.layers.Reshape(target_shape=(28, 28, 1)),
  tf.keras.layers.Conv2D(filters=4, kernel_size=(3, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
  tf.keras.layers.Conv2D(filters=8, kernel_size=(3, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
  tf.keras.layers.Conv2D(filters=16, kernel_size=(3, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(10)
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape (Reshape)            (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 26, 26, 4)         40        
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 4)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 8)         296       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 8)           0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 16)          1168      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 1, 1, 16)          0

## Salvado de modelo en formato SavedModel

El formato `SavedModel` se usa, entre otras cosas, como formato _de facto_ para servir a traves de TensorFlow Serve. El salvado y la carga de estos modelos es muy parecida a la del método `save` de los modelos de Keras.

Vamos a salvar nuestro modelo recién creado.

In [6]:
tf.keras.models.save_model(model, 'base_model.pb')

2021-10-31 22:26:17.548100: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: base_model.pb/assets


Para cargarlo, es prácticamente igual de sencillo

In [7]:
model = tf.keras.models.load_model('base_model.pb')
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape (Reshape)            (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 26, 26, 4)         40        
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 4)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 8)         296       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 8)           0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 16)          1168      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 1, 1, 16)          0

## Entrenamiento y cálculo de valores para el modelo base

Vamos a entrenar el modelo para usar los valores de _loss_, _accuracy_, tiempo de ejecución y tamaño del fichero como valores de referencia. Para ello entrenaremos primero el modelo base

In [8]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE);

Epoch 1/10
469/469 [==============================] - 2s 2ms/step - loss: 0.9727 - accuracy: 0.6872
Epoch 2/10
469/469 [==============================] - 1s 2ms/step - loss: 0.3407 - accuracy: 0.8965
Epoch 3/10
469/469 [==============================] - 1s 2ms/step - loss: 0.2672 - accuracy: 0.9173
Epoch 4/10
469/469 [==============================] - 1s 2ms/step - loss: 0.2313 - accuracy: 0.9286
Epoch 5/10
469/469 [==============================] - 1s 2ms/step - loss: 0.2069 - accuracy: 0.9359
Epoch 6/10
469/469 [==============================] - 1s 2ms/step - loss: 0.1914 - accuracy: 0.9412
Epoch 7/10
469/469 [==============================] - 1s 2ms/step - loss: 0.1800 - accuracy: 0.9446
Epoch 8/10
469/469 [==============================] - 1s 2ms/step - loss: 0.1693 - accuracy: 0.9483
Epoch 9/10
469/469 [==============================] - 1s 2ms/step - loss: 0.1622 - accuracy: 0.9495
Epoch 10/10
469/469 [==============================] - 1s 2ms/step - loss: 0.1546 - accuracy: 0.9525

Ahora obtenemos los valores. No es la mejor forma de calcular los tiempos, pero nos puede dar una ligera idea.

In [9]:
start = time.time()
base_loss, base_acc = model.evaluate(X_test, y_test, verbose=0)
base_time = time.time() - start
tf.keras.models.save_model(model, 'model.pb', include_optimizer=False)
base_size = model_size("model.pb")

print(f'Saved base model to: "model.pb" ({base_size:.4} MiB)')

INFO:tensorflow:Assets written to: model.pb/assets
Saved base model to: "model.pb" (0.153 MiB)


## Poda del modelo

La poda de pesos significa eliminar los valores innecesarios en los tensores que almacenan los pesos ajustados durante los entrenamiento. Prácticamente estamos poniendo a cero los valores de los parámetros de la red neuronal para eliminar lo que estimamos que son conexiones innecesarias entre las capas de una red neuronal. La siguiente figura lo representa de manera muy explícita.

| <img src="https://miro.medium.com/max/1400/0*iNI8Oc80Eunm8NgI" alt="Pruning example" width="500"> | 
|:--:| 
| *Ejemplo de poda. Fuente: [miro.medium.com](https://miro.medium.com)* |

Ahora aplicaremos la poda a todo el modelo. En este caso en concreto usamos la función `prune_low_magnitude` que tratará de encontrar pesos suficientemente pequeños como para convertirlos en 0 y así convertir la matriz de pesos en una matriz dispersa (menos espacio y más rapidez en los cálculos)

En la [guía completa](https://www.tensorflow.org/model_optimization/guide/pruning/comprehensive_guide.md) se explica con más detalle las diferentes opciones tanto de objetos de configuración (e.g. `PolynomialDecay`, un detector de pesos pequeños similar al `LearningRateDecay` para el factor de aprendizaje) como de poda selectiva por capas.

**NOTA**: La función `prune_low_magnitude` reescribe el modelo, y por tanto necesita una recompilación del modelo.

In [10]:
pruned_model = tf.keras.models.load_model('base_model.pb')
pruned_model = tfmot.sparsity.keras.prune_low_magnitude(pruned_model)

pruned_model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)
pruned_model.summary()

pruned_model.fit(
    X_train,
    y_train,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks = [tfmot.sparsity.keras.UpdatePruningStep()]
);

/home/blazaid/.local/share/virtualenvs/deep-learning-99e-ingI/lib/python3.8/site-packages/keras/engine/base_layer.py:2223: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
prune_low_magnitude_reshape  (None, 28, 28, 1)         1         
_________________________________________________________________
prune_low_magnitude_conv2d ( (None, 26, 26, 4)         78        
_________________________________________________________________
prune_low_magnitude_max_pool (None, 13, 13, 4)         1         
_________________________________________________________________
prune_low_magnitude_conv2d_1 (None, 11, 11, 8)         586       
_________________________________________________________________
prune_low_magnitude_max_pool (None, 5, 5, 8)           1         
_________________________________________________________________
prune_low_magnitude_conv2d_2 (None, 3, 3, 16)          2322      
_________________________________________________________________
prune_low_magnitude_max_pool (None, 1, 1, 16)          1

Ahora tomaremos los valores del modelo podado para compararlos con los valores del modelo base.

In [11]:
start = time.time()
pruned_loss, pruned_accuracy = pruned_model.evaluate(X_test, y_test, verbose=0)
pruned_time = time.time() - start
tf.keras.models.save_model(pruned_model, 'pruned_model.pb', include_optimizer=False)
pruned_size = model_size("pruned_model.pb")

print(f'Saved pruned model to: "pruned_model.pb" ({pruned_size:.4} MiB)')

INFO:tensorflow:Assets written to: pruned_model.pb/assets


INFO:tensorflow:Assets written to: pruned_model.pb/assets


Saved pruned model to: "pruned_model.pb" (0.832 MiB)


De hecho, vamos a compararlos

In [12]:
print(f'Base loss:   {base_loss:.4}; base accuracy:   {base_acc:.4}; model size: {base_size:.4}; inference time: {base_time:.4}')
print(f'Pruned loss: {pruned_loss:.4}; pruned accuracy: {pruned_accuracy:.4}; model size: {pruned_size:.4}; inference time: {pruned_time:.4}')

Base loss:   0.1456; base accuracy:   0.957; model size: 0.153; inference time: 0.5205
Pruned loss: 0.2011; pruned accuracy: 0.9394; model size: 0.832; inference time: 0.372


En cuestion de espacio no sale muy bien parado. Esto es debido porque el modelo contiene información de parámetros que controlan las ramas a podar (en forma de _floats_ de 32 bits o 64 bits, dependiendo de donde se esté ejecutando). La _cuantización_, entre otras cosas, se encargará de comprimir al máximo estas ramas.

En cuestión de tiempo, sin embargo, parece que ha mejorado algo. No es para tirar cohetes, pero algo es algo. Sigamos adelante.

## Cuantización

El primer paso para la cuantización es habilitar el entrenamiento para ella, similar a lo que hacíamos con la poda.

In [13]:
quantized_model = tf.keras.models.load_model('base_model.pb')
quantized_model = tfmot.quantization.keras.quantize_model(quantized_model)

quantized_model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)
quantized_model.summary()

quantized_model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE);

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
quantize_layer (QuantizeLaye (None, 28, 28)            3         
_________________________________________________________________
quant_reshape (QuantizeWrapp (None, 28, 28, 1)         1         
_________________________________________________________________
quant_conv2d (QuantizeWrappe (None, 26, 26, 4)         51        
_________________________________________________________________
quant_max_pooling2d (Quantiz (None, 13, 13, 4)         1         
_________________________________________________________________
quant_conv2d_1 (QuantizeWrap (None, 11, 11, 8)         315       
_________________________________________________________________
quant_max_pooling2d_1 (Quant (None, 5, 5, 8)           1         
_________________________________________________________________
quant_conv2d_2 (QuantizeWrap (None, 3, 3, 16)          1

Y ahora, cogemos los valores para comparar con los anteriores.

In [14]:
start = time.time()
quantized_loss, quantized_accuracy = quantized_model.evaluate(X_test, y_test, verbose=0)
quantized_time = time.time() - start
tf.keras.models.save_model(quantized_model, 'quantized_model.pb', include_optimizer=False)
quantized_size = model_size("quantized_model.pb")

print(f'Saved quantized model to: "quantized_model.pb" ({quantized_size:.4} MiB)')

INFO:tensorflow:Assets written to: quantized_model.pb/assets


INFO:tensorflow:Assets written to: quantized_model.pb/assets


Saved quantized model to: "quantized_model.pb" (0.4978 MiB)


Vamos a comparar:

In [15]:
print(f'Base:      loss: {base_loss:.4}; accuracy:   {base_acc:.4}; size: {base_size:.4}; time: {base_time:.4}')
print(f'Pruned:    loss: {pruned_loss:.4}; accuracy: {pruned_accuracy:.4}; size: {pruned_size:.4}; time: {pruned_time:.4}')
print(f'Quantized: loss: {quantized_loss:.4}; accuracy: {quantized_accuracy:.4}; size: {quantized_size:.4}; time: {quantized_time:.4}')

Base:      loss: 0.1456; accuracy:   0.957; size: 0.153; time: 0.5205
Pruned:    loss: 0.2011; accuracy: 0.9394; size: 0.832; time: 0.372
Quantized: loss: 0.1302; accuracy: 0.9631; size: 0.4978; time: 0.4859


Ha reducido del tamaño, y si bien es algo más lento que el podado, no deja de ser mejor que un modelo normal sin optimizar. Además, este es el paso previo a optimizaciones de cuantización de verdad, porque aún no hemos cuantizado como tal (por ejemplo, los pesos siguen siendo de 32 bits o de 64 bits). Nosotros nos limitaremos a la optimización por defecto para la exportación a [TensorFlow Lite](https://www.tensorflow.org/lite), pero en la guía [_Quantization aware training comprehensive guide_](https://www.tensorflow.org/model_optimization/guide/quantization/training_comprehensive_guide) se exponen difeerntes métodos de optimización según plataformas objetivo.

Por cierto, para cargar modelos entrenados para cuantización, es necesario hacerlo dentro del contexto `quantize_scope`:

In [16]:
with tfmot.quantization.keras.quantize_scope():
    quantized_model = tf.keras.models.load_model("quantized_model.pb")
    quantized_model.compile(
        optimizer='adam',
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=['accuracy']
    )

### Cuantización por defecto del modelo base

En este paso usaremos el convertidos a modelos de TensorFlow Lite. De esta manera conseguiremos unos modelos a priori más rápidos y compactos, aunque con cierta pérdida en la exactitud de las predicciones.

En este caso nos centraremos únicamente en el espacio ocupado, ya que el trabajo con estos modelos es más tedioso.

In [17]:
tflite_model = tf.keras.models.load_model('model.pb')

converter = tf.lite.TFLiteConverter.from_keras_model(tflite_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tf_lite_model = converter.convert()
with open('tf_lite_model.pb', 'wb') as f:
    f.write(tf_lite_model)
tflite_size = model_size('tf_lite_model.pb')

print(f'Base model size:      {base_size:.4}')
print(f'Pruned model size:    {pruned_size:.4}')
print(f'Quantized model size: {quantized_size:.4}')
print(f'TFLite model size:    {tflite_size:.4}')

INFO:tensorflow:Assets written to: /tmp/tmptanac92u/assets


INFO:tensorflow:Assets written to: /tmp/tmptanac92u/assets


Base model size:      0.153
Pruned model size:    0.832
Quantized model size: 0.4978
TFLite model size:    0.00769


2021-10-31 22:27:06.421257: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:351] Ignored output_format.
2021-10-31 22:27:06.421275: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:354] Ignored drop_control_dependency.


### Cuantización por defecto en modelos marcados para cuantización

Por último, al igual que antes, veamos en este caso los tamaños involucrados

In [18]:
q_tflite_model = tf.keras.models.load_model('quantized_model.pb')

converter = tf.lite.TFLiteConverter.from_keras_model(q_tflite_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
q_tflite_model = converter.convert()
with open('q_tflite_model.pb', 'wb') as f:
    f.write(q_tflite_model)
q_tflite_size = model_size('q_tflite_model.pb')

print(f'Base model size:             {base_size:.4}')
print(f'Pruned model size:           {pruned_size:.4}')
print(f'Quantized model size:        {quantized_size:.4}')
print(f'TFLite model size:           {tflite_size:.4}')
print(f'Quantized TFLite model size: {q_tflite_size:.4}')

INFO:tensorflow:Assets written to: /tmp/tmpd5d050w9/assets


INFO:tensorflow:Assets written to: /tmp/tmpd5d050w9/assets


Base model size:             0.153
Pruned model size:           0.832
Quantized model size:        0.4978
TFLite model size:           0.00769
Quantized TFLite model size: 0.009331


2021-10-31 22:27:07.593395: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:351] Ignored output_format.
2021-10-31 22:27:07.593416: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:354] Ignored drop_control_dependency.
